In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import psycopg2

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Change as needed, if we pick a different model
from sklearn.svm import SVC 


In [2]:
# # based on:
# # https://pythontic.com/pandas/serialization/postgresql
# # https://medium.com/@alestamm/importing-data-from-a-postgresql-database-to-a-pandas-dataframe-5f4bffcd8bb2
# # https://naysan.ca/2020/05/31/postgresql-to-pandas/
# # https://towardsdatascience.com/python-and-postgresql-how-to-access-a-postgresql-database-like-a-data-scientist-b5a9c5a0ea43

# # Load data from database
# # Create an engine instance
# alchemyEngine   = create_engine('postgresql+psycopg2://test:@127.0.0.1', pool_recycle=3600);

# engine = create_engine('postgresql+psycopg2:INSERT HERE')


# # Connect to PostgreSQL server
# connection = engine.connect()


# # Read data from PostgreSQL database table and load into a DataFrame instance

df = pd.read_sql("select * from \"TABLENAME\"", connection)

df.head()

 

# Close the database connection

connection.close()

In [3]:
# # Separate data into features and target
# # NEED TO CLARIFY TARGET(S)

# y = df['TARGET NAME']
# X = dr.drop(columns = 'TARGET NAME', 1)

In [4]:
# # Split dataset into training and testing

# # Do we need a random_state here? Maybe?
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, stratify = y)
# X_train.shape

In [6]:
# # Train the model
# model.fit(X_train, y_train)

In [7]:
# # Create predictions
# y_pred = model.predict(X_test)
# results = pd.DataFrame({
#     "Prediction": y_pred,
#     "Actual": y_test
# }).reset_index(drop = True)
# results.head()

In [8]:
# # Assess accuracy score
# accuracy_score(y_test, y_pred)

In [9]:
# # Generate confusion matrix
# confusion_matrix(y_test, y_pred)

In [ ]:
# # Print classification report
# print(classification_report(y_test, y_pred)